<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import mip
except ImportError:
    import sys
    !{sys.executable} -m pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


# Create Player dataset

In [2]:
# file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
# df = pd.read_csv(file_path)#, on_bad_lines='skip')
# df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

# def extract_numbers(s):
#     if pd.isna(s):
#         return None
#     numbers = re.findall(r'\d+', str(s))
#     if numbers:
#         return int(numbers[0])
#     return None

# num_of_players = len(df) #250

# projections_df = df[['Player', 'Bye', 'POS', 'AVG']].head(num_of_players).copy()
# projections_df['Bye'] = projections_df['Bye'].apply(extract_numbers)
# projections_df['Bye'] = projections_df['Bye'].fillna(0).astype(int)

# projections_df.info()
# print(projections_df.value_counts('POS'))
# print(projections_df.head(20))

In [26]:
weekly_projections_url = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/projection_data/2024/weekly_projections_2024.csv'
weekly_projections = pd.read_csv(weekly_projections_url)
print(weekly_projections.columns)
weekly_projections_colums = ['player', 'position', 'team', 'points', 'dropoff', 'week', 'year']
weekly_projections = weekly_projections[weekly_projections_colums].copy()
weekly_projections = weekly_projections.rename(columns={'player': 'Player'})
weekly_projections = weekly_projections.rename(columns={'position': 'POS'})
weekly_projections.head(10)

Index(['Unnamed: 0', 'player', 'position', 'team', 'points', 'sd_pts',
       'dropoff', 'floor', 'ceiling', 'points_vor', 'floor_vor', 'ceiling_vor',
       'rank', 'floor_rank', 'ceiling_rank', 'position_rank', 'tier', 'adp',
       'aav', 'uncertainty', 'week', 'year'],
      dtype='object')


,Player,POS,team,points,dropoff,week,year
0,Christian McCaffrey,RB,SF,21.4,4.5000,1,2024
1,Tyreek Hill,WR,MIA,21.5,1.6300,1,2024
2,Amon-Ra St. Brown,WR,DET,19.8,1.3500,1,2024
3,Josh Allen,QB,BUF,24.6,2.7900,1,2024
4,Bijan Robinson,RB,ATL,16.9,0.0282,1,2024
5,Breece Hall,RB,NYJ,16.9,0.5460,1,2024
6,Nick Chubb,RB,CLE,16.3,0.0619,1,2024
7,Saquon Barkley,RB,PHI,16.3,0.2390,1,2024
8,Justin Jefferson,WR,MIN,18.5,0.1440,1,2024
9,Jahmyr Gibbs,RB,DET,16.0,0.0310,1,2024


In [21]:
f = weekly_projections.pivot_table(
    index=['player', 'position'],   # Use player and position as row indices
    columns='week',                 # Weeks become columns
    values='points',                # Points are the values to fill
    aggfunc='first'                 # In case of duplicates, take the first
)

# Rename columns to 'Week X'
f.columns = [f'Week {col}' for col in f.columns]
f['TTL'] = f.sum(axis=1)
f = f.sort_values('TTL', ascending=False)
f = f.reset_index()
f = f.rename(columns={'position': 'POS'})
f = f.rename(columns={'player': 'Player'})
f = f.fillna(0)

head_size = 15
f.head(head_size)

,Player,POS,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,...,Week 10,Week 11,Week 12,Week 13,Week 14,Week 15,Week 16,Week 17,Week 18,TTL
0,Lamar Jackson,QB,19.0,21.5,20.0,19.8,21.0,22.9,21.7,21.8,...,23.2,21.8,21.9,21.4,0.0,22.7,22.3,22.3,23.6,368.7
1,Josh Allen,QB,24.6,22.8,22.9,22.0,21.6,18.7,21.6,21.9,...,21.3,21.4,0.0,22.1,22.8,23.2,25.8,24.7,0.0,359.6
2,Jayden Daniels,QB,16.7,18.7,17.5,20.9,20.8,21.0,22.6,19.2,...,21.1,18.4,20.3,19.0,0.0,20.6,18.4,20.9,21.4,339.0
3,Ja'Marr Chase,WR,16.3,17.4,19.4,18.3,17.2,19.3,18.4,20.2,...,21.1,19.8,0.0,20.2,20.7,21.5,22.4,21.9,22.6,336.6
4,Joe Burrow,QB,18.4,17.9,19.4,18.7,18.1,19.2,18.1,18.7,...,19.8,19.0,0.0,19.9,21.3,20.3,21.5,20.0,20.4,329.7
5,Justin Jefferson,WR,18.5,17.6,19.4,19.0,18.0,0.0,19.9,19.2,...,20.8,18.3,17.9,17.7,18.9,18.6,18.3,20.0,21.8,324.2
6,Jalen Hurts,QB,21.8,22.7,21.2,20.6,0.0,20.7,19.4,19.7,...,22.0,21.8,21.5,21.5,22.5,20.7,21.9,0.0,0.0,320.4
7,Bijan Robinson,RB,16.9,19.4,17.6,17.5,16.0,16.0,17.5,17.8,...,20.0,18.3,0.0,18.8,17.7,19.3,20.3,18.2,21.1,311.2
8,Amon-Ra St. Brown,WR,19.8,18.6,17.9,16.9,0.0,17.7,16.7,18.8,...,17.3,18.1,18.7,18.2,17.7,17.8,19.2,18.4,20.1,309.4
9,Kyler Murray,QB,17.4,18.8,19.5,20.9,17.1,17.7,17.6,17.6,...,16.9,0.0,17.6,16.8,17.3,18.6,18.7,17.2,17.4,304.9


In [28]:
players_url = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
players = pd.read_csv(players_url)
print(players.columns)

players_colums = ['Player', 'Team', 'Bye', 'POS']
players = players[players_colums].copy()

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

players['Bye'] = players['Bye'].apply(extract_numbers)
players['Bye'] = players['Bye'].fillna(0).astype(int)
players['POS'] = players['POS'].str.replace('\d+', '', regex=True)

print(players.value_counts('POS'))

players

Index(['Rank', 'Player', 'Team', 'Bye', 'POS', 'ESPN', 'Sleeper', 'NFL',
       'RTSports', 'FFC', 'AVG'],
      dtype='object')
POS
WR     335
RB     224
TE     166
QB     127
K       64
DST     32
Name: count, dtype: int64


,Player,Team,Bye,POS
0,Christian McCaffrey,SF,9,RB
1,CeeDee Lamb,DAL,7,WR
2,Tyreek Hill,MIA,6,WR
3,Bijan Robinson,ATL,12,RB
4,Breece Hall,NYJ,12,RB
...,...,...,...,...
943,Ryan Miller,TB,11,WR
944,Mitchell Tinsley,CIN,12,WR
945,La'Michael Pettway,NaN,0,TE
946,Anthony Gould,IND,14,WR


# Create custom projections
Maybe delete later when real projections available

In [23]:
# # Anzahl der Wochen
# number_of_weeks = 17

# # Wochen-Spaltennamen
# weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# # Skalierungsfunktion
# def projection_base(avg, pos, max_val=22, min_val=7, k=75, c=1.5):
#     proj = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
#     if pos == "QB":
#         proj += 6  # QB-Bonus
#     elif pos == "K":
#         proj -= 2  # K-Penalty
#     elif pos == "DST":
#         proj -= 4  # DST-Penalty
#     return proj

# final_projections = []
# for _, row in projections_df.iterrows():
#     base_score = projection_base(row['AVG'], row['POS'])

#     # Erstellen der weekly projections
#     weekly_proj = []
#     for week in range(number_of_weeks):
#         # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
#         if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
#             weekly_proj.append(0.0)  # Projektion auf 0 setzen
#         else:
#             weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

#     final_projections.append(weekly_proj)

# # Projektionen in DataFrame einfügen
# f = projections_df[["Player", "Bye", "POS", "AVG"]].copy()
# for i, col in enumerate(weekly_columns):
#     f[col] = [proj[i] for proj in final_projections]
# f["TTL"] = f[weekly_columns].sum(axis=1)
# f['dropoff'] = (f.sort_values(['POS','TTL'], ascending=[True, False]).groupby('POS')['TTL'].diff(-1).fillna(0.0))


# # Stichprobe
# probe = 30
# print(f'Länge von f: {len(f)}')
# print(f'head({probe}) of f:')
# print()
# # print(f.head(probe))
# f.head(probe)

# Optimization

In [29]:
# ==========================
# 1. PARAMETERS
# ==========================

players = players['Player'].copy().tolist()
positions = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks = list(range(1, 18))
pos = dict(zip(weekly_projections['Player'], weekly_projections['POS']))
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}
week_cols = [col for col in f.columns if col.startswith("Week_")]

# Build f_dict: weekly projections and dropoff for each player
f_dict = {
    row['Player']: {
        **{int(week.replace("Week_", "")): row[week] for week in week_cols},
        'dropoff': row['dropoff']
    }
    for _, row in f.iterrows()
}

beta = {t: 140.0 for t in weeks}
gamma = {"QB": 2, "RB": 3, "WR": 3, "TE": 2, "K": 1, "DST": 1}
alpha, lambda_0, lambda_1, lambda_2, lambda_3 = 1.0, 1, 100, 200, 100
df_sorted = weekly_projections.sort_values("AVG").reset_index(drop=True)
df_sorted["Rank"] = df_sorted.index + 1
topk_pct = 0.0025
min_pos_req = pos_limit.copy()  # Minimum roster requirements per position

# ==========================
# 2. INITIALIZATION (TEAMS, DRAFT ORDER, OPPONENT PICK)
# ==========================

num_teams = 12
teams = [f"Team {i+1}" for i in range(num_teams)]
DM_team = "Team 3"
num_rounds = 15

# Snake draft order
draft_order = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
    # 1) Sort remaining players by rank
    rem = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(rem) * topk_pct))

    # 2) Calculate deficits per position (min requirement minus current roster)
    deficits = {
        j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in min_pos_req
    }
    needed = [j for j, d in deficits.items() if d > 0]

    # 3) If deficits exist, pick from candidates in those positions
    if needed:
        candidates = [p for p in rem if pos[p] in needed]
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    # 4) Fallback: pick randomly from top-k overall
    return random.choice(rem[:topk])

# ==========================
# 3. DRAFT INITIALIZATION
# ==========================

rosters = {tm: [] for tm in teams}   # Dict: team -> list of drafted players
available = set(players)             # Set of available players
draft_log = []                       # List to store draft results

# ==========================
# 4. MAIN DRAFT LOOP
# ==========================

for pick_idx, team in enumerate(draft_order, start=1):

    # ---- 4.1: Update remaining player ranking ----
    rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, "AVG"].item())
    Rk = {p: i+1 for i, p in enumerate(rem)}
    player_vars = set(rem) | set(rosters[team])
    picks_remaining = num_rounds - len(rosters[team])

    # ---- 4.2: DM-Team (your team) picks via MIP ----
    if team == DM_team:
        m = Model(sense=maximize, solver_name="CBC")

        # --- Decision variables ---
        # y[i]: 1 if player i is drafted by DM, 0 otherwise
        y = {i: m.add_var(var_type=BINARY, name=f"y_{i}") for i in player_vars}
        # x[i, t]: share of player i's points used in week t (continuous)
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}") for i in rem for t in weeks}
        # z[t]: 1 if DM wins in week t, 0 otherwise
        z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

        # --- Objective function ---
        m.objective = (
            lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in rem for t in weeks) # Total points
            + lambda_1 * xsum(z[t] for t in weeks[:15])                         # Early win indicator
            + lambda_2 * xsum(z[t] for t in weeks[15:])                         # Late win indicator
            + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in rem)         # Dropoff bonus
        )

        # --- Constraints ---
        # Fix previous picks (already drafted players must remain picked)
        for p in rosters[team]:
            m += y[p] == 1

        # Enforce that exactly the remaining picks are made
        m += xsum(y[i] for i in rem) == picks_remaining

        # Enforce minimum requirements for each position (relative to already drafted players)
        for pos_name, req in min_pos_req.items():
            already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
            need = max(0, req - already_satisfied)
            m += xsum(y[i] for i in rem if pos[i] == pos_name) >= need

        # Weekly lineup constraints and position limits
        for j in positions:
        #     m += xsum(y[i] for i in rem if pos[i] == j) >= gamma[j]  # Minimum number per position
            for t in weeks:
                m += xsum(x[i, t] for i in rem if pos[i] == j) <= pos_limit[j]  # Weekly lineup limit

        # Only drafted players can be in the weekly lineup
        for i in rem:
            for t in weeks:
                m += x[i, t] <= y[i]

        # Win indicator constraints
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in rem) / beta[t]

        # Robust draft constraint (to simulate uncertainty in opponent picks)
        n_k = pick_idx
        for future_pick in range(
            pick_idx + 1,
            pick_idx + picks_remaining * len(teams),
            len(teams)
        ):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        # --- Solve the MIP model ---
        m.optimize()
        # Check for infeasibility
        if m.num_solutions == 0:
            raise RuntimeError(f"No feasible solution at pick {pick_idx}. Check constraints and player pool.")

        # Extract chosen player for this pick
        chosen = [i for i in rem if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f"No feasible pick at {pick_idx}")
        pick = min(chosen, key=lambda i: Rk[i])

    # ---- 4.3: Opponent pick (simple heuristic) ----
    else:
        current_round = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster=rosters[team],
            available=available,
            Rk=Rk,
            min_pos_req=pos_limit,
            topk_pct=dynamic_topk_pct
        )

    # ---- 4.4: Update rosters and draft log ----
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        "Pick": pick_idx,
        "Team": team,
        "Player": pick,
        "Round": (pick_idx - 1) // len(teams) + 1,
        "POS": pos[pick]
    })

# ==========================
# 5. CREATE DRAFT DATAFRAME
# ==========================

df_draft = pd.DataFrame(draft_log)
print(df_draft.head(1 + num_teams * 2))


KeyError: 'dropoff'

In [ ]:
merged = pd.merge(df_draft, f[['Player', 'TTL']], on='Player', how='left')

# Replace missing TTLs (e.g., für Spieler ohne Projection) mit 0
merged['TTL'] = merged['TTL'].fillna(0)

# Group by Team and sum TTL to get total projection per team
team_ttl_proj = merged.groupby('Team')['TTL'].sum().reset_index()
team_ttl_proj = team_ttl_proj.rename(columns={'TTL': 'TTL_proj'})
team_ttl_proj

In [ ]:
# # --- 1. Parameter ---
# players        = projections_df['Player'].tolist()
# positions      = {"QB", "RB", "WR", "TE", "K", "DST"}
# weeks          = list(range(1,18))
# pos            = dict(zip(projections_df['Player'], projections_df['POS']))
# pos_limit      = {"QB":1,"RB":2,"WR":2,"TE":1,"K":1,"DST":1}
# week_cols = [col for col in f.columns if col.startswith("Week_")]
# f_dict = {
#     row['Player']: {**{int(week.replace("Week_", "")): row[week] for week in week_cols}, 'dropoff': row['dropoff']}
#     for _, row in f.iterrows()
# }
# beta           = {t:140.0 for t in weeks}
# gamma          = {"QB":2,"RB":3,"WR":3,"TE":2,"K":1,"DST":1}
# alpha, lambda_0, lambda_1, lambda_2, lambda_3 = 1.0, 1, 100, 150, 25
# df_sorted = projections_df.sort_values("AVG").reset_index(drop=True)
# df_sorted["Rank"] = df_sorted.index + 1
# topk_pct = 0.005
# min_pos_req = pos_limit


# # --- 2. Teams, DM-Team und Snake-Draft ---
# num_teams      = 12
# teams          = [f"Team {i+1}" for i in range(num_teams)]
# DM_team        = "Team 3"
# num_rounds     = 15
# draft_order    = []
# for rnd in range(num_rounds):
#     order = teams if rnd % 2 == 0 else teams[::-1]
#     draft_order += order

# def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
#     # 1) verbleibende Spieler neu sortieren
#     rem  = sorted(available, key=lambda p: Rk[p])
#     topk = max(1, int(len(rem) * topk_pct))

#     # 2) Defizite je Position (Mindestsoll minus aktueller Bestand)
#     deficits = {
#         j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
#         for j in min_pos_req
#     }
#     needed = [j for j, d in deficits.items() if d > 0]

#     # 3) solange Defizite bestehen, aus allen rem dieser Position picken
#     if needed:
#         # Kandidaten aller benötigten Positionen
#         candidates = [p for p in rem if pos[p] in needed]
#         # begrenze auf Top-k, falls mehr Kandidaten vorhanden
#         pool = candidates[:topk] if len(candidates) >= topk else candidates
#         if pool:
#             return random.choice(pool)

#     # 4) Fallback: zufällig aus Top-k aller Positionen
#     return random.choice(rem[:topk])

# # --- 3. Initialisierung ---
# rosters        = {tm: [] for tm in teams}
# available      = set(players)
# draft_log      = []

# # --- 4. Optimierung über alle Picks ---
# for pick_idx, team in enumerate(draft_order, start=1):
#     # 4.1 Ranking der verbleibenden Spieler aktualisieren
#     rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player==p,"AVG"].item())
#     Rk  = {p: i+1 for i,p in enumerate(rem)}
#     player_vars = set(rem) | set(rosters[team])
#     picks_remaining = num_rounds - len(rosters[team])


#     # 4.2 DM-Pick via MIP
#     if team == DM_team:
#         m = Model(sense=maximize, solver_name="CBC")

#         # Entscheidungsvariablen
#         y = {i: m.add_var(var_type=BINARY, name=f"y_{i}")
#               for i in player_vars}
#         x = {(i,t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}")
#              for i in rem for t in weeks}
#         z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

#         # Objective Function
#         m.objective = (
#             lambda_0 * xsum(f_dict[i][t]*x[i,t] for i in rem for t in weeks)
#           + lambda_1 * xsum(z[t] for t in weeks[:15])
#           + lambda_2 * xsum(z[t] for t in weeks[15:])
#           + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in rem)
#         )

#         # Constraints
#         # Fixiere vergangene Picks
#         for p in rosters[team]:
#             m += y[p] == 1

#         # Exakte Anzahl verbleibender Picks
#         m += xsum(y[i] for i in rem) == picks_remaining

#         # Mindestanforderungen relativ zu schon gezogenen Spielern
#         for pos_name, req in min_pos_req.items():
#             already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
#             need = max(0, req - already_satisfied)
#             m += xsum(y[i] for i in rem if pos[i] == pos_name) >= need

#         # Position‐ und Roster‐Constraints
#         for j in positions:
#             m += xsum(y[i] for i in rem if pos[i]==j) >= gamma[j]
#             for t in weeks:
#                 m += xsum(x[i,t] for i in rem if pos[i]==j) <= pos_limit[j]

#         # # Roster Anforderungen
#         # for pos_name, req in min_pos_req.items():
#         #     m.add_constr(
#         #         xsum(y[i] for i in player_vars if pos.get(i) == pos_name) >= req,
#         #         name=f"min_roster_{pos_name}")
#         # # maximale Picks pro Team
#         # m += xsum(y[i] for i in rem) <= num_rounds

#         # nur gedraftete Spieler in der Week‐Lineup
#         for i in rem:
#             for t in weeks:
#                 m += x[i,t] <= y[i]

#         # Win‐Indicator
#         for t in weeks:
#             m += z[t] <= xsum(f_dict[i][t]*x[i,t] for i in rem) / beta[t]

#         # Robuste Draft‐Constraint
#         n_k = pick_idx
#         for future_pick in range(pick_idx+1, pick_idx + (num_rounds - len(rosters[team]))*len(teams), len(teams)):
#             top_cut = int(alpha*(future_pick - n_k))
#             if top_cut > 0:
#                 top_players = [i for i,r in Rk.items() if r <= top_cut]
#                 m += xsum(y[i] for i in top_players) <= ( (future_pick-n_k) // len(teams) )

#         m.optimize()
#         # if m.num_solutions == 0:
#         #     raise RuntimeError(f"No feasible solution at pick {pick_idx}. Check constraints and player pool.")

#         # chosen = [i for i in rem if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
#         # if not chosen:
#         #     raise RuntimeError(f"No feasible pick at {pick_idx}")
#         # pick = min(chosen, key=lambda i: Rk[i])

#         # gewählten Spieler extrahieren
#         chosen = [i for i in rem if y[i].x >= 0.99 and i not in rosters[team]]
#         if not chosen:
#             raise RuntimeError(f"No feasible pick at {pick_idx}")
#         pick = min(chosen, key=lambda i: Rk[i])

#     # 4.3 Gegner-Pick: zufällig aus Top-5 verbleibend
#     else:
#         pick = opponent_pick(roster=rosters[team], available=available, Rk=Rk, min_pos_req=pos_limit, topk_pct=topk_pct)

#     # 4.4 Update
#     rosters[team].append(pick)
#     available.remove(pick)
#     draft_log.append({
#         "Pick": pick_idx, "Team": team, "Player": pick,
#         "Round": (pick_idx-1)//len(teams)+1, "POS": pos[pick]
#     })

# # --- 5. Ergebnis als DataFrame ---
# df_draft = pd.DataFrame(draft_log)
# print(df_draft.head(1+num_teams*2))

In [ ]:
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)

In [ ]:
position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
position_counts_df

In [ ]:
result_dfs = {}

for team in df_draft['Team'].unique():
    team_df = df_draft[df_draft['Team'] == team].copy()
    team_df = team_df.sort_values(by='Pick')

    # Optional: Spalten anpassen, wenn nur bestimmte Infos gewünscht sind
    team_df['Pick Info'] = team_df.apply(lambda row: f"Round {row['Round']} Pick {row['Pick']}", axis=1)
    result_dfs[team] = team_df[['Player', 'Pick Info', 'POS']]  # oder andere gewünschte Spalten

for team, df in result_dfs.items():
    print(f"=== {team} ===")
    print(df)
    print()
